In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os, re
import shutil
import random
import dill

from sklearn.model_selection import train_test_split
import skimage
from skimage.io import imread, imsave

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.losses import categorical_crossentropy
from keras.optimizers import adam, sgd
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## Load Data

* ### init

In [2]:
# create df
img_path = os.path.join(os.getcwd(), "data/raw_data/resize-mixed")
names = [f.split(".")[0] for f in os.listdir(img_path) if any(c.isdigit() for c in f)]
labels = [re.sub(r'\d+', '', name) for name in names]
paths = [os.path.join(img_path, f+".jpg") for f in names]
dict_df = pd.DataFrame(zip(names, paths, labels), columns=['name','path', 'label'], dtype=str)
dict_df.head()

,name,path,label
0,metal296,/Users/loaner/Documents/github/trash-classifie...,metal
1,plastic391,/Users/loaner/Documents/github/trash-classifie...,plastic
2,cardboard233,/Users/loaner/Documents/github/trash-classifie...,cardboard
3,cardboard227,/Users/loaner/Documents/github/trash-classifie...,cardboard
4,plastic385,/Users/loaner/Documents/github/trash-classifie...,plastic


* ### data augmentation

In [12]:
# image augmentation 
from skimage import transform
def rotate(image_array: np.ndarray):
    '''rotate image with a random degree (between left_degree and right_degree)'''
    degree_limit = 30
    rotate_degree = random.uniform(-degree_limit, degree_limit)
    return transform.rotate(image_array, rotate_degree)*255

def add_noise(image_array: np.ndarray):
    '''add noise to image'''
    return skimage.util.random_noise(image_array)*255

def hor_flip(image_array: np.ndarray):
    '''flipping pixels horizontally'''
    return image_array[:, ::-1]

def ver_flip(image_array: np.ndarray):
    '''flipping pixels vertically'''
    return image_array[::-1, :]

transform_methods = [rotate, add_noise, hor_flip, ver_flip]

In [4]:
def create_more_trash(target_num, base_file_paths, count_for_name):
    num_transformed = 0
    aug_img_df = pd.DataFrame(columns=dict_df.columns)
    
    base_folder = os.path.join('data/raw_data/resized', 'trash')
    aug_img_path = 'data/raw_data/resize_split/train/trash'
    aug_path = 'data/raw_data/aug_data'

    # delete if aug folder already exists (re-aug for every training set)
    if not os.path.exists(aug_path):
        os.mkdir(aug_path)
    else:
        for f in os.listdir(aug_path):
            os.remove(os.path.join(aug_path, f))
    
    while num_transformed <= target_num:
        rand_path = random.choice(base_file_paths)
        # base_img = dill.load(open(rand_path, 'rb'))
        base_img = imread(rand_path)

        new_img = transform_methods[random.randint(0,3)](base_img).astype(np.uint8)
        num_transformed += 1
        
        new_img_name = "trash{}".format(str(count_for_name + num_transformed))
        new_img_path = os.path.join(aug_path, new_img_name)
        
        aug_img_df = aug_img_df.append({"name":new_img_name, "path":new_img_path, "label": "trash"},ignore_index=True)
        dill.dump(new_img, open(new_img_path, 'wb'))
        imsave(os.path.join(aug_img_path, new_img_name+".jpg"),new_img)
    return aug_img_df

* ### split

In [5]:
def split_and_aug(data_df, test_size):

    X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(data_df, data_df['label'], \
                                                                    test_size=test_size)

    # data augmentation prep
    trash_base_df = X_train_df[X_train_df['label']=='trash']
    trash_base_paths = trash_base_df['path'].tolist()
    if len(trash_base_df[~trash_base_df['name'].str.contains('trash')])!=0:
        raise "label incorrect"  # sanity check
    else:
        avg_numb = int((len(X_train_df)- len(trash_base_paths))/5)
        trash_aug_numb = avg_numb - len(trash_base_paths)
        aug_trash_df = create_more_trash(trash_aug_numb, trash_base_paths, 137)
    
    # load the augmented data into training set
    X_train_df = X_train_df.append(aug_trash_df)
    y_train_df = y_train_df.append(aug_trash_df['label'])
    
    # copy image to the resize_split folder
    for index, row in X_train_df.iterrows():
        dest_dir = os.path.join('data/raw_data/resize_split', 'train', row['label'])
        shutil.copy(row['path'], dest_dir)
        row['path'] = os.path.join(dest_dir, row['name']+".jpg")
    
    for index, row in X_test_df.iterrows():
        dest_dir = os.path.join('data/raw_data/resize_split', 'test', row['label'])
        shutil.copy(row['path'], dest_dir)
        row['path'] = os.path.join(dest_dir, row['name']+".jpg")
    
    return X_train_df, X_test_df, y_train_df, y_test_df

* ### image generator

In [6]:
def generate_img_tensor(train_dir, test_dir, im_width, im_height, batch_size):
    
    im_gen = ImageDataGenerator(featurewise_center=False,
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 rotation_range=15, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 horizontal_flip=True, 
                                 vertical_flip=True)


    train_batch = im_gen.flow_from_directory(train_dir, target_size=(im_width, im_height), batch_size=batch_size)
    test_batch = im_gen.flow_from_directory(test_dir,target_size=(im_width, im_height), batch_size=batch_size)
    
    return train_batch, test_batch

## Build Model

In [8]:
def cnn_model(im_w, im_h):
    
    model = Sequential()

    model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(im_w,im_h,3)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same'))
    model.add(Dropout(0.25))

#     model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
#     model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
#     model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
#     model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
              
    model.summary()
              
    return model


In [9]:
def make_pred(model, img_path, img_w, img_h):
    pic = imread(img_path)
    pic = cv2.resize(pic, (img_w, img_h))
    pic = np.expand_dims(pic, axis=0)
    y_pred = model.predict_classes(pic)

## Train and Predict

* ### prep

In [10]:
train_path = 'data/raw_data/resize_split/train'
test_path = 'data/raw_data/resize_split/test'

img_w = 384
img_h = 512
num_classes = 6
batch_size = 50

In [13]:
X_train_df, X_test_df, y_train_df, y_test_df = split_and_aug(dict_df, 0.2)

In [14]:
train_tensor, test_tensor = generate_img_tensor(train_path, test_path, img_w, img_h, batch_size)

Found 2287 images belonging to 6 classes.
Found 506 images belonging to 6 classes.


* ### train

In [15]:
model = cnn_model(img_w, img_h)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 384, 512, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 192, 256, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 192, 256, 16)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 786432)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               402653696 
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)              

In [16]:
checkpoint = ModelCheckpoint('cnn_1', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [17]:
model.compile(loss='categorical_crossentropy', optimizer=adam(lr=1.0e-4), metrics=['accuracy'])

In [ ]:
model = model.fit_generator(train_tensor,  
                            validation_data=test_tensor,
                            steps_per_epoch=len(X_train_df) // batch_size,
                            validation_steps=len(X_test_df) // batch_size,
                            epochs=20, 
                            verbose=1, 
                            callbacks=[checkpoint])

Epoch 1/20
34/45 [=====================>........] - ETA: 2:32 - loss: 1199.3241 - accuracy: 0.1700

In [ ]:
model.save('my_model.h5')